In [3]:
import torch
from sklearn.metrics import f1_score
import dgl
from utils import load_data, EarlyStopping
import torch.nn.functional as F
from model_hetero import HAN, HAN_freebase
from tools import evaluate_results_nc
import numpy as np
from augment import Augmenter

def consis_loss(logps, temp):
    ps = [torch.exp(p) for p in logps]
    sum_p = 0.
    for p in ps:
        sum_p = sum_p + p
    avg_p = sum_p/len(ps)
    #p2 = torch.exp(logp2)
    
    sharp_p = (torch.pow(avg_p, 1./temp) / torch.sum(torch.pow(avg_p, 1./temp), dim=1, keepdim=True)).detach()
    loss = 0.
    for p in ps:
        loss += torch.mean((p-sharp_p).pow(2).sum(1))
    loss = loss/len(ps)
    return args.rate * loss

def score(logits, labels):
    _, indices = torch.max(logits, dim=1)
    prediction = indices.long().cpu().numpy()
    labels = labels.cpu().numpy()

    accuracy = (prediction == labels).sum() / len(prediction)
    micro_f1 = f1_score(labels, prediction, average='micro')
    macro_f1 = f1_score(labels, prediction, average='macro')

    return accuracy, micro_f1, macro_f1


def evaluate(model, g, features, labels, mask, loss_func):
    model.eval()
    with torch.no_grad():
        logits,_ = model(g, features)
    loss = loss_func(logits[mask], labels[mask])
    accuracy, micro_f1, macro_f1 = score(logits[mask], labels[mask])

    return loss, accuracy, micro_f1, macro_f1


def main(args):
    g, features, labels, num_classes, train_idx, val_idx, test_idx, train_mask, \
    val_mask, test_mask, meta_paths, dl = load_data(args.dataset, feat_type=0)
#     print('features>>>>>>>>> ',len(features),len(features[0]))
    
    unlabel_idx = test_idx

    if hasattr(torch, 'BoolTensor'):
        train_mask = train_mask.bool()
        val_mask = val_mask.bool()
        test_mask = test_mask.bool()

    dataset = features.to(args.device)
    labels = labels.to(args.device)
    train_mask = train_mask.to(args.device)
    val_mask = val_mask.to(args.device)
    test_mask = test_mask.to(args.device)
    if args.dataset == 'Freebase':
        # Add a fc layer to calculate sparse
        model = HAN_freebase(
            meta_paths=meta_paths,
            in_size=features.shape[1],
            hidden_size=args.hidden_units,
            out_size=num_classes,
            num_heads=args.num_heads,
            dropout=args.dropout).to(args.device)
    else:
        model = HAN(
            meta_paths=meta_paths,
            in_size=features.shape[1],
            hidden_size=args.hidden_units,
            out_size=num_classes,
            num_heads=args.num_heads,
            dropout=args.dropout).to(args.device)

    g = g.to(args.device)

    stopper = EarlyStopping(patience=args.patience)
    loss_fcn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr,
                                 weight_decay=args.weight_decay)

    augmenter = Augmenter(args=args, dl=dl, is_label=labels)
        
    for epoch in range(args.num_epochs):
        loss_u_list = []
        model.train()
        print('features>>>>',features.size())
#         print('****************')
        logits,_ = model(g, features)
#         loss = loss_fcn(logits[train_mask], labels[train_mask])
        loss_l = loss_fcn(logits[train_mask], labels[train_mask])
        loss_u_list.append(torch.log_softmax(logits[unlabel_idx], dim=-1))
        # ------ 节点增强 ------
        aug_features = augmenter.node_aug_heterograph_dblp(dl, features, unlabel_idx, args.cf_k)        
        logits1,_ = model(g, aug_features)
        loss_u_list.append(torch.log_softmax(logits1[unlabel_idx], dim=-1))
        # ------ 结构增强 ------
#         aug_g = augmenter.triangle_aug_heterograph()
#         logits2,_ = model(aug_g, features)
#         loss_u_list.append(torch.log_softmax(logits2[unlabel_idx], dim=-1))

        loss_u = consis_loss(loss_u_list, temp=args.tem)
        loss = loss_l + loss_u

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc, train_micro_f1, train_macro_f1 = score(logits[train_mask], labels[train_mask])
        val_loss, val_acc, val_micro_f1, val_macro_f1 = evaluate(model, g, features, labels, val_mask, loss_fcn)
        early_stop = stopper.step(val_loss.data.item(), val_acc, model)

        print('Epoch {:d} | Train Loss {:.4f} | Train Micro f1 {:.4f} | Train Macro f1 {:.4f} | '
              'Val Loss {:.4f} | Val Micro f1 {:.4f} | Val Macro f1 {:.4f}'.format(
            epoch + 1, loss.item(), train_micro_f1, train_macro_f1, val_loss.item(), val_micro_f1, val_macro_f1))

        if early_stop:
            break

    stopper.load_checkpoint(model)
    test_loss, test_acc, test_micro_f1, test_macro_f1 = evaluate(model, g, features, labels, test_mask, loss_fcn)
    print('Test loss {:.4f} | Test Micro f1 {:.4f} | Test Macro f1 {:.4f}'.format(
        test_loss.item(), test_micro_f1, test_macro_f1))

    model.eval()
    with torch.no_grad():
        logits,test_embeddings = model(g, features)
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_embeddings[test_idx].cpu().numpy(), dl.labels_test['data'][dl.labels_test['mask']].argmax(axis=1), num_classes=dl.labels_test['num_classes'])


In [7]:
if __name__ == '__main__':
    import argparse

    from utils import setup

    parser = argparse.ArgumentParser('HAN')
    parser.add_argument('-s', '--seed', type=int, default=1,
                        help='Random seed')
    parser.add_argument('-ld', '--log-dir', type=str, default='results',
                        help='Dir for saving training results')
    parser.add_argument('--hetero', action='store_true',
                        help='Use metapath coalescing with DGL\'s own dataset')
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--dataset', type=str, default='DBLP',
                        choices=['DBLP', 'ACM', 'Freebase', 'JD', 'ogbm'])
    parser.add_argument('--device', type=str, default='cuda:0')
    parser.add_argument('--num_epochs', type=int, default=50)
    parser.add_argument('--patience', type=int, default=100)
    parser.add_argument('--hidden_units', type=int, default=64)
    parser.add_argument('--num_heads', type=int, default=[8])
    parser.add_argument('--dropout', type=float, default=0)
    parser.add_argument('--lr', type=float, default=1e-2)
    parser.add_argument('--weight_decay', type=float, default=1e-3)
    parser.add_argument('--cf_k', type=int, default=2)
    parser.add_argument('--tem', type=float, default=0.2)
    parser.add_argument('--rate', type=float, default=0.2)

    args = parser.parse_args(args=[])
    print(args)
    main(args)

Namespace(cf_k=2, dataset='DBLP', device='cuda:0', dropout=0, hetero=False, hidden_units=64, log_dir='results', lr=0.01, num_epochs=50, num_heads=[8], num_layers=2, patience=100, rate=0.2, seed=1, tem=0.2, weight_decay=0.001)
loading data:
loading data >>>>>>>>>>>>>>>>>>>>>
0 {'0': 4057}
1 {'0': 4057, '1': 14328}
2 {'0': 4057, '1': 14328, '2': 7723}
3 {'0': 4057, '1': 14328, '2': 7723, '3': 20}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
sample_num: 119783
features>>>> torch.Size([4057, 20])
self.dataset.links.items()>>>>>>  dict_items([(0, (1, 0)), (1, (1, 2)), (2, (1, 3)), (3, (0, 1)), (4, (2, 1)), (5, (3, 1))])


/ossfs/workspace/HGB/NC/benchmark/methods/HAN/augment.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


Epoch 1 | Train Loss 1.3824 | Train Micro f1 0.3404 | Train Macro f1 0.1994 | Val Loss 1.4008 | Val Micro f1 0.1393 | Val Macro f1 0.0674
features>>>> torch.Size([4057, 20])
self.dataset.links.items()>>>>>>  dict_items([(0, (1, 0)), (1, (1, 2)), (2, (1, 3)), (3, (0, 1)), (4, (2, 1)), (5, (3, 1))])


/ossfs/workspace/HGB/NC/benchmark/methods/HAN/augment.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


EarlyStopping counter: 1 out of 100
Epoch 2 | Train Loss 1.3638 | Train Micro f1 0.3568 | Train Macro f1 0.1431 | Val Loss 1.4445 | Val Micro f1 0.1339 | Val Macro f1 0.0590
features>>>> torch.Size([4057, 20])
self.dataset.links.items()>>>>>>  dict_items([(0, (1, 0)), (1, (1, 2)), (2, (1, 3)), (3, (0, 1)), (4, (2, 1)), (5, (3, 1))])


...

EarlyStopping counter: 1 out of 100
Epoch 48 | Train Loss 0.3291 | Train Micro f1 0.9073 | Train Macro f1 0.8945 | Val Loss 0.1938 | Val Micro f1 0.9290 | Val Macro f1 0.9198
features>>>> torch.Size([4057, 20])
self.dataset.links.items()>>>>>>  dict_items([(0, (1, 0)), (1, (1, 2)), (2, (1, 3)), (3, (0, 1)), (4, (2, 1)), (5, (3, 1))])


/ossfs/workspace/HGB/NC/benchmark/methods/HAN/augment.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


EarlyStopping counter: 2 out of 100
Epoch 49 | Train Loss 0.3236 | Train Micro f1 0.9085 | Train Macro f1 0.8966 | Val Loss 0.1951 | Val Micro f1 0.9372 | Val Macro f1 0.9289
features>>>> torch.Size([4057, 20])
self.dataset.links.items()>>>>>>  dict_items([(0, (1, 0)), (1, (1, 2)), (2, (1, 3)), (3, (0, 1)), (4, (2, 1)), (5, (3, 1))])


/ossfs/workspace/HGB/NC/benchmark/methods/HAN/augment.py:305: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neigh_fes = torch.tensor(neigh_fes/2.0, dtype=torch.float)


EarlyStopping counter: 3 out of 100
Epoch 50 | Train Loss 0.3135 | Train Micro f1 0.9108 | Train Macro f1 0.8969 | Val Loss 0.1922 | Val Micro f1 0.9344 | Val Macro f1 0.9265
Test loss 0.2945 | Test Micro f1 0.8950 | Test Macro f1 0.8882
SVM test
Macro-F1: 0.901598~0.013047 (0.8), 0.896023~0.006566 (0.6), 0.898292~0.005673 (0.4), 0.891385~0.006084 (0.2)
Micro-F1: 0.905986~0.012749 (0.8), 0.900968~0.006089 (0.6), 0.903169~0.005372 (0.4), 0.896831~0.005617 (0.2)
K-means test
NMI: 0.630917~0.000000
ARI: 0.660766~0.000000


In [3]:
a = ([1,2,3],[2,3,4])
a[0]

[1, 2, 3]